In [10]:
import requests
import time
import csv
from datetime import datetime

In [11]:
# Constants
COINGECKO_API_BASE_URL = "https://api.coingecko.com/api/v3"
INPUT_FILE = "id.csv"  # File containing 100 coin IDs, one per line
OUTPUT_FILE = "coin_min_max_dates.csv"
API_CALL_DELAY = 10  # Delay in seconds between API calls to respect rate limits

In [12]:
def read_coin_ids(file_path):
    with open(file_path, 'r') as file:
        coin_ids = [line.strip() for line in file if line.strip()]
    return coin_ids

In [13]:
def get_coin_data(coin_id):
    url = f"{COINGECKO_API_BASE_URL}/coins/{coin_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error for {coin_id}: {http_err}")
    except Exception as err:
        print(f"Error fetching data for {coin_id}: {err}")
    return None


In [14]:
def get_min_max_dates(coin_data):
    # Get genesis date as min date
    genesis_date = coin_data.get('genesis_date')
    if genesis_date:
        min_date = datetime.strptime(genesis_date, "%Y-%m-%d").date()
    else:
        # If genesis_date is not available, you might need to fetch historical data
        min_date = None

    # Get the last_updated field as max date
    last_updated = coin_data.get('last_updated')
    if last_updated:
        max_date = datetime.strptime(last_updated, "%Y-%m-%dT%H:%M:%S.%fZ").date()
    else:
        max_date = datetime.today().date()

    return min_date, max_date


In [15]:
def main():
    coin_ids = read_coin_ids(INPUT_FILE)
    print(f"Found {len(coin_ids)} coin IDs.")

    results = []

    for idx, coin_id in enumerate(coin_ids, start=1):
        print(f"Processing {idx}/{len(coin_ids)}: {coin_id}")
        coin_data = get_coin_data(coin_id)
        if coin_data:
            min_date, max_date = get_min_max_dates(coin_data)
            if min_date and max_date:
                results.append({
                    'coin_id': coin_id,
                    'min_date': min_date.isoformat(),
                    'max_date': max_date.isoformat()
                })
                print(f" - Min Date: {min_date}, Max Date: {max_date}")
            else:
                print(f" - Incomplete date information for {coin_id}.")
        else:
            print(f" - Failed to retrieve data for {coin_id}.")

        time.sleep(API_CALL_DELAY)  # Respect API rate limits

    # Write results to CSV
    with open(OUTPUT_FILE, 'w', newline='') as csvfile:
        fieldnames = ['coin_id', 'min_date', 'max_date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for row in results:
            writer.writerow(row)

    print(f"Results saved to {OUTPUT_FILE}")

In [16]:
if __name__ == "__main__":
    main()

Found 101 coin IDs.
Processing 1/101: id
HTTP error for id: 404 Client Error: Not Found for url: https://api.coingecko.com/api/v3/coins/id
 - Failed to retrieve data for id.
Processing 2/101: bitcoin
 - Min Date: 2009-01-03, Max Date: 2024-11-24
Processing 3/101: ethereum
 - Min Date: 2015-07-30, Max Date: 2024-11-24
Processing 4/101: tether
 - Incomplete date information for tether.
Processing 5/101: solana
 - Incomplete date information for solana.
Processing 6/101: binancecoin
 - Min Date: 2017-07-08, Max Date: 2024-11-24
Processing 7/101: ripple
 - Incomplete date information for ripple.
Processing 8/101: dogecoin
 - Min Date: 2013-12-08, Max Date: 2024-11-24
Processing 9/101: usd-coin
 - Incomplete date information for usd-coin.
Processing 10/101: cardano
 - Incomplete date information for cardano.
Processing 11/101: staked-ether
 - Incomplete date information for staked-ether.
Processing 12/101: tron
 - Min Date: 2017-08-28, Max Date: 2024-11-24
Processing 13/101: avalanche-2
 - 